In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [6]:
base = os.path.abspath('data')
#base = os.path.abspath('/kaggle/input/liverpool-ion-switching/')
#base = 'https://github.com/hjort/ai-labs/raw/master/jupyter/ion-channels/data/'

train = pd.read_csv(os.path.join(base + '/train.csv.bz2'), #nrows=5000,
                    dtype={'time': np.float32, 'signal': np.float32, 'open_channels': 'uint8'})
test  = pd.read_csv(os.path.join(base + '/test.csv.bz2'), #nrows=5000,
                    dtype={'time': np.float32, 'signal': np.float32})
submission = pd.read_csv(os.path.join(base + '/sample_submission.csv.bz2'), #nrows=5000,
                    dtype={'time': np.float32, 'open_channels': 'uint8'})

In [7]:
train.head()

,time,signal,open_channels
0,0.0001,-2.7600,0
1,0.0002,-2.8557,0
2,0.0003,-2.4074,0
3,0.0004,-3.1404,0
4,0.0005,-3.1525,0


In [8]:
train.tail()

,time,signal,open_channels
4999995,499.999603,2.9194,7
4999996,499.999695,2.6980,7
4999997,499.999786,4.5164,8
4999998,499.999908,5.6397,9
4999999,500.000000,5.3792,9


In [9]:
train.loc[49000:50010,:]

,time,signal,open_channels
49000,4.9001,-2.9378,0
49001,4.9002,-2.7240,0
49002,4.9003,-2.5879,0
49003,4.9004,-2.3940,0
49004,4.9005,-2.3708,0
49005,4.9006,-3.1857,0
49006,4.9007,-2.9674,0
49007,4.9008,-2.5549,0
49008,4.9009,-2.9567,0
49009,4.9010,-3.2541,0


In [10]:
train.shape

(5000000, 3)

In [11]:
train['open_channels'].min()

0

In [12]:
train_time = train['time'].values

In [13]:
train_time_0 = train_time[:50000]

In [14]:
train_time_0 = list(train_time_0) * 100

In [15]:
len(train_time_0)

5000000

In [16]:
train['time'] = train_time_0

In [17]:
test.head()

,time,signal
0,500.000092,-2.6498
1,500.000214,-2.8494
2,500.000305,-2.8600
3,500.000397,-2.4350
4,500.000488,-2.6155


In [18]:
test.tail()

,time,signal
1999995,699.999573,-2.9092
1999996,699.999695,-2.7422
1999997,699.999817,-2.8285
1999998,699.999878,-2.9092
1999999,700.000000,-2.7422


In [19]:
test.shape

(2000000, 2)

In [20]:
train_time_0 = train_time[:50000]
train_time_0 = list(train_time_0) * 40
test['time'] = train_time_0

In [21]:
n_groups = 100
train["group"] = 0
for i in range(n_groups):
    ids = np.arange(i * 50000, (i + 1) * 50000)
    train.loc[ids, "group"] = i
    
n_groups = 40
test["group"] = 0
for i in range(n_groups):
    ids = np.arange(i * 50000, (i + 1) * 50000)
    test.loc[ids, "group"] = i
    
train['signal_2'] = 0
test['signal_2'] = 0

n_groups = 100
for i in range(n_groups):
    sub = train[train.group == i]
    signals = sub.signal.values
    imax, imin = math.floor(np.max(signals)), math.ceil(np.min(signals))
    signals = (signals - np.min(signals)) / (np.max(signals) - np.min(signals))
    signals = signals * (imax - imin)
    train.loc[sub.index, "signal_2"] = [0,] + list(np.array(signals[:-1]))
    
n_groups = 40
for i in range(n_groups):
    sub = test[test.group == i]
    signals = sub.signal.values
    imax, imin = math.floor(np.max(signals)), math.ceil(np.min(signals))
    signals = (signals - np.min(signals)) / (np.max(signals) - np.min(signals))
    signals = signals * (imax - imin)
    test.loc[sub.index, "signal_2"] = [0,] + list(np.array(signals[:-1]))

In [22]:
X = train[['time', 'signal_2']].values
y = train['open_channels'].values

In [28]:
X[:-10]

array([[9.99999975e-05, 0.00000000e+00],
       [1.99999995e-04, 6.22782230e-01],
       [3.00000014e-04, 5.56713879e-01],
       ...,
       [4.99879980e+00, 7.66238165e+00],
       [4.99889994e+00, 6.94801044e+00],
       [4.99900007e+00, 6.52027893e+00]])

In [27]:
y[:-10]

array([0, 0, 0, ..., 8, 8, 8], dtype=uint8)

In [31]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit(X, y)

print(reg.coef_)
print(reg.intercept_)

model = reg

[-0.00418771  0.95029944]
-0.5976528230868845


In [54]:
from sklearn import linear_model
reg = linear_model.RidgeCV(alphas=np.logspace(-6, 6, 13))
%time reg.fit(X, y)
reg.alpha_

1.0

In [70]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0)
%time clf.fit(X, y)

model = clf

/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


CPU times: user 11min 17s, sys: 11.7 s, total: 11min 29s
Wall time: 11min 32s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [74]:
model = clf

In [75]:
model.predict(X)

array([0, 0, 0, ..., 3, 3, 8], dtype=uint8)

In [23]:
from sklearn.linear_model import Ridge
model = Ridge()
model.fit(X, y)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [56]:
train_preds = model.predict(X)
train_preds

array([-5.9765e-01, -5.8241e-03, -6.8609e-02, ...,  4.6150e+00,
        6.1542e+00,  7.1051e+00])

In [57]:
train_preds = np.clip(train_preds, 0, 10)
train_preds.mean()

2.7368485468118107

In [58]:
train_preds = train_preds.astype(int)

In [76]:
X_test = test[['time', 'signal_2']].values

In [77]:
submission.head()

,time,open_channels
0,500.0001,0
1,500.0002,0
2,500.0003,0
3,500.0004,0
4,500.0005,0


In [78]:
submission.shape

(2000000, 2)

In [79]:
X_test.shape

(2000000, 2)

In [80]:
test_preds = model.predict(X_test)
test_preds = np.clip(test_preds, 0, 10)
test_preds = test_preds.astype('uint8')
submission['open_channels'] = test_preds

In [81]:
test_preds.mean()

1.13359

In [82]:
submission.head()

,time,open_channels
0,500.0001,0
1,500.0002,0
2,500.0003,0
3,500.0004,0
4,500.0005,0


In [83]:
np.set_printoptions(precision=4)

In [84]:
submission.time.values[:20]

array(['500.0001', '500.0002', '500.0003', '500.0004', '500.0005',
       '500.0006', '500.0007', '500.0008', '500.0009', '500.0010',
       '500.0011', '500.0012', '500.0013', '500.0014', '500.0015',
       '500.0016', '500.0017', '500.0018', '500.0019', '500.0020'],
      dtype=object)

In [85]:
submission.to_csv('ion-channels-submission-logreg.csv', index=False)

In [86]:
!gzip -9 ion-channels-submission-*.csv